In [116]:
import load_data_tf as load_data

"""
    Ensure that 'patches_with_cloud_and_shadow.csv' and 'patches_with_seasonal_snow.csv' are in the locations specified
    (download from GCP)
"""
filter_files = ['../patches_with_cloud_and_shadow.csv', '../patches_with_seasonal_snow.csv'] # replace with your path

data_dir = "../SmallEarthNet" # put the path to the dataset here

"""
    This will create the dataset and create a split file (if it doesn't exist) at ./smallearthnet.pkl and label count
    data at ./label_counts.pkl (takes ~10-15s).
    
    RECOMMENDED: download ./smallearthnet.pkl and ./label_counts.pkl from the GCP bucket.
"""
support_size = 8
label_subset_size = 3
meta_dataset = load_data.MetaBigEarthNetTaskDataset(data_dir=data_dir, filter_files=filter_files, 
                                                    support_size=support_size, label_subset_size=label_subset_size,
                                                    split_save_path="smallearthnet.pkl", 
                                                    split_file="smallearthnet.pkl")

%load_ext autoreload
%autoreload 2

  1%|          | 93/9607 [00:00<00:10, 923.02it/s]

Reloading train-val-test split cache from smallearthnet.pkl


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
import time
meta_dataset.support_size = 16
meta_dataset.label_subset_size = 3
start = time.time()
X, y, y_debug = meta_dataset.sample_batch(1, mode='permutation')
print("Batch generated in {:.4f}s".format(time.time() - start))
print(y_debug)

from utils import convert_to_powerset
print(np.argmax(convert_to_powerset(y), axis=-1))

[13 12 33]
1 (array([12]),)
3 (array([13]),)
0 (array([33]),)
1 (array([12]),)
3 (array([13]),)
0 (array([33]),)
1 (array([12]),)
3 (array([13]),)
0 (array([33]),)
1 (array([12]),)
3 (array([13]),)
0 (array([33]),)
1 (array([12]),)
3 (array([13]),)
0 (array([33]),)
1 (array([12]),)
Batch generated in 0.0474s
[[[12. -1. -1.]
  [13. -1. -1.]
  [33. -1. -1.]
  [12. -1. -1.]
  [13. -1. -1.]
  [33. -1. -1.]
  [12. -1. -1.]
  [13. -1. -1.]
  [33. -1. -1.]
  [12. -1. -1.]
  [13. -1. -1.]
  [33. -1. -1.]
  [12. -1. -1.]
  [13. -1. -1.]
  [33. -1. -1.]
  [12. -1. -1.]]]
[[0 1 3 0 1 3 0 1 3 0 1 3 0 1 3 0]]


In [138]:
a = [i for i, (_, lbl) in enumerate(meta_dataset.dataset) if lbl[22]]

SystemError: <built-in function Open> returned a result with an error set

In [26]:
from tqdm.notebook import tqdm
from collections import defaultdict

# build up label-index-dict (used for permutation-shot sampling)
label_indices_dict = defaultdict(set)
for i, (_, lbl) in enumerate(tqdm(meta_dataset.dataset)):
    for class_id in np.where(lbl)[0]:
        label_indices_dict[class_id].add(i)
    

In [96]:
import random
from functools import reduce 
from itertools import cycle
import operator as op

random.seed(420)
key_indices = range(len(label_indices_dict))
label_subset_size = 3

classes = random.sample(key_indices, label_subset_size)
print("Classes:", *classes)
powerset_labels = range(1 << label_subset_size)

support_indices = []
powerset_label_index_list = []
for perm in powerset_labels:
    if not perm: continue
    perm_string = '0{}b'.format(label_subset_size)
    intersect_perm = [c for i, c in zip(format(perm, perm_string), classes) if i == '1']
    exclude_perm = [c for i, c in zip(format(perm, perm_string), classes) if i == '0']
    valid_indices = reduce(op.and_, [label_indices_dict[i] for i in intersect_perm])
    excluded = [label_indices_dict[i] for i in exclude_perm]
    if len(excluded):
        valid_indices -= reduce(op.and_, [label_indices_dict[i] for i in exclude_perm])
    powerset_label_index_list.append(valid_indices)
    
index_list_indices = list(range(len(powerset_label_index_list)))
random.shuffle(index_list_indices)
index_set_iter = cycle(index_list_indices)
while len(support_indices) < 16:
    i = next(index_set_iter)
    index_set = powerset_label_index_list[i]
    if len(index_set):
        di = random.sample(index_set, 1)[0]
        img, label = meta_dataset.dataset[di]
        support_indices.append(di)

print(support_indices)

Classes: 1 23 17
[8633, 3528, 8663, 9548, 3197, 8572, 3596, 8228, 3595, 2572, 6734, 3528, 8337, 9547, 1680, 8632]


In [76]:
powerset_label_index_list

[{3527, 3528, 3595, 3596, 6471, 9442, 9465, 9547, 9548, 9559, 9570, 9582},
 {6733,
  6734,
  6779,
  6815,
  7636,
  7637,
  8172,
  8173,
  8174,
  8194,
  8213,
  8228,
  8229,
  8245,
  8246,
  8250,
  8258,
  8262,
  8263,
  8264,
  8265,
  8266,
  8267,
  8268,
  8274,
  8279,
  8280,
  8281,
  8286,
  8298,
  8303,
  8304,
  8310,
  8318,
  8319,
  8320,
  8329,
  8330,
  8331,
  8332,
  8336,
  8337,
  8338,
  8347,
  8352,
  8353,
  8354,
  8414,
  8415,
  8484,
  8485,
  8510,
  8511,
  8539,
  8540,
  8545,
  8570,
  8571,
  8572,
  8602,
  8604,
  8605,
  8630,
  8631,
  8632,
  8633,
  8634,
  8635,
  8659,
  8660,
  8661,
  8662,
  8663,
  8681,
  8684,
  8685,
  8686,
  8687,
  8688,
  8689,
  8690,
  8691,
  8695,
  8708,
  8720},
 set(),
 {0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  28,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,


In [47]:
class_perm = [c for i, c in zip(format(1, '0{}b'.format(label_subset_size)), classes) if i == '1']

[50]

In [48]:
reduce(op.and_, [label_indices_dict[i]  for i in class_perm])

NameError: name 'reduce' is not defined